# Resumen

Actualmente, **sin criterios de racionalizacion** tendriamos que cerrar una brecha de **5,743 plazas** y costaria **S/ 89,832,006**. Sin embargo si se usan los siguientes **criterios de ordenamiento**:

1) Nombrados excedentes se reubican dentro de su UGEL

2) Las plazas no nombradas excedentes se reubican a nivel nacional para cubrir los requerimientos

La brecha seria de **2,976 plazas** con un costo **S/46,550,592** lo que implica un **ahorro** de **2,767 plazas** equivalentes a **S/ 43,281,414**

In [1]:
#Libraries
import os 
import pandas as pd
import numpy as np

os.chdir('D:\Brecha-no-docente') # Directorio de trabajo

In [2]:
df = pd.read_stata('Resultados\Brecha con costo.dta')

C:\Users\USUARIO\Anaconda3\lib\site-packages\pandas\io\stata.py:1457: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


In [3]:
identificacion = ['d_dpto', 'region', 'pliego', 'codue', 'unidadejecutora', 'codooii', 'nombentidad', 'tipo_entidad']
datos = ['pers_limp_mant_n', 'pers_limp_mant_c', 'pers_limp_mant', 'opt_pers_limp_mant', 'exd_pers_limp_mant', 'req_pers_limp_mant',
        'pers_vigilancia_n', 'pers_vigilancia_c', 'pers_vigilancia', 'opt_pers_vigilancia', 'exd_pers_vigilancia', 'req_pers_vigilancia', 
         'coord_adm_ie_n', 'coord_adm_ie_c', 'coord_adm_ie', 'opt_coord_adm_ie', 'exd_coord_adm_ie', 'req_coord_adm_ie',
         'oficinista_n', 'oficinista_c', 'oficinista', 'opt_oficinista', 'exd_oficinista', 'req_oficinista',
         'secretario_n', 'secretario_c', 'secretario', 'opt_secretario', 'exd_secretario', 'req_secretario'
        ]

In [4]:
df_ugel = df.groupby(identificacion)[datos].sum()
df_ugel.reset_index(inplace = True)

for x in datos:
    df_ugel[x] = df_ugel[x].astype(int)

In [5]:
personal = ['pers_limp_mant','pers_vigilancia', 'coord_adm_ie', 'oficinista', 'secretario']

# Desagrego el personal excedente
for x in personal:
    df_ugel[f'exd_{x}_c'] = df_ugel[[f'exd_{x}',f'{x}_c']].min(axis=1)  
    df_ugel[f'exd_{x}_n'] = df_ugel[f'exd_{x}'] - df_ugel[f'exd_{x}_c']

# Brecha sin racionalizacion

En esta brecha se financia todos los requerimientos

In [6]:
# Calculo de la brecha
costo_oficinista = 12*1200+600+12*(1200*0.09)
costo_secretario = 12*1400+600+12*(1400*0.09)
costo_otros      = 12*1150+600+12*(1150*0.09)

brecha_sr = df_ugel.copy()
for x in personal:
    brecha_sr[f'brecha_sr_{x}'] = brecha_sr[f'req_{x}']
    if x == 'oficinista':
        brecha_sr[f'costo_brecha_sr_{x}'] = brecha_sr[f'brecha_sr_{x}']*costo_oficinista
    elif x == 'secretario':
        brecha_sr[f'costo_brecha_sr_{x}'] = brecha_sr[f'brecha_sr_{x}']*costo_secretario
    else:
        brecha_sr[f'costo_brecha_sr_{x}'] = brecha_sr[f'brecha_sr_{x}']*costo_otros


In [53]:
# Resumen - region
region_sr = brecha_sr.groupby('d_dpto').sum()

sr_costos = []
sr_personal = []
for x in personal:
    sr_personal.append(f'brecha_sr_{x}')
    sr_costos.append(f'costo_brecha_sr_{x}')

In [60]:
# Brecha en plazas
region_sr[sr_personal].style.format({'brecha_sr_pers_limp_mant'  : '{:20,.0f}', 
                                    'brecha_sr_pers_vigilancia' : '{:20,.0f}', 
                                    'brecha_sr_coord_adm_ie'    : '{:20,.0f}',
                                    'brecha_sr_oficinista'      : '{:20,.0f}', 
                                    'brecha_sr_secretario'      : '{:20,.0f}'})

,brecha_sr_pers_limp_mant,brecha_sr_pers_vigilancia,brecha_sr_coord_adm_ie,brecha_sr_oficinista,brecha_sr_secretario
d_dpto,,,,,
AMAZONAS,71,220,160,39,2
ANCASH,86,503,360,169,16
APURIMAC,51,155,152,37,4
AREQUIPA,127,457,346,83,2
AYACUCHO,70,254,181,90,5
CAJAMARCA,262,580,455,131,11
CALLAO,79,490,210,113,28
CUSCO,269,508,431,113,10
HUANCAVELICA,22,102,104,30,1


In [61]:
# Brecha en costos
region_sr[sr_costos].style.format({'costo_brecha_sr_pers_limp_mant'  : 'S/{:20,.0f}', 
                        'costo_brecha_sr_pers_vigilancia' : 'S/{:20,.0f}', 
                        'costo_brecha_sr_coord_adm_ie'    : 'S/{:20,.0f}',
                        'costo_brecha_sr_oficinista'      : 'S/{:20,.0f}', 
                        'costo_brecha_sr_secretario'      : 'S/{:20,.0f}'})

,costo_brecha_sr_pers_limp_mant,costo_brecha_sr_pers_vigilancia,costo_brecha_sr_coord_adm_ie,costo_brecha_sr_oficinista,costo_brecha_sr_secretario
d_dpto,,,,,
AMAZONAS,"S/ 1,110,582","S/ 3,441,240","S/ 2,502,720","S/ 635,544","S/ 37,824"
ANCASH,"S/ 1,345,212","S/ 7,867,926","S/ 5,631,120","S/ 2,754,024","S/ 302,592"
APURIMAC,"S/ 797,742","S/ 2,424,510","S/ 2,377,584","S/ 602,952","S/ 75,648"
AREQUIPA,"S/ 1,986,534","S/ 7,148,394","S/ 5,412,132","S/ 1,352,568","S/ 37,824"
AYACUCHO,"S/ 1,094,940","S/ 3,973,068","S/ 2,831,202","S/ 1,466,640","S/ 94,560"
CAJAMARCA,"S/ 4,098,204","S/ 9,072,360","S/ 7,117,110","S/ 2,134,776","S/ 208,032"
CALLAO,"S/ 1,235,718","S/ 7,664,580","S/ 3,284,820","S/ 1,841,448","S/ 529,536"
CUSCO,"S/ 4,207,698","S/ 7,946,136","S/ 6,741,702","S/ 1,841,448","S/ 189,120"
HUANCAVELICA,"S/ 344,124","S/ 1,595,484","S/ 1,626,768","S/ 488,880","S/ 18,912"


In [57]:
# Resumen
resumen_sr_personal = list(brecha_sr[sr_personal].sum())
resumen_sr_costos = list(brecha_sr[sr_costos].sum())
personal_resumen = ['Limpieza y mantenimiento','Vigilantes','Coordinador administrativo','Oficinista','Secretario']
dict_resumen_sr = {'Perfiles': personal_resumen, 'Plazas': resumen_sr_personal, 'Costos': resumen_sr_costos} 
    
resumen_sr = pd.DataFrame(dict_resumen_sr)
resumen_sr.loc['5'] = resumen_sr.sum()
resumen_sr.iloc[5, resumen_sr.columns.get_loc('Perfiles')] = 'Total'

print('En resumen: ')
resumen_sr.style.format({'Plazas':'{:20,.0f}','Costos':'S/{:20,.0f}' })

En resumen: 


,Perfiles,Plazas,Costos
0,Limpieza y mantenimiento,"5,743","S/ 89,832,006"
1,Vigilantes,"14,281","S/ 223,383,402"
2,Coordinador administrativo,"8,355","S/ 130,688,910"
3,Oficinista,"4,236","S/ 69,029,856"
4,Secretario,477,"S/ 9,021,024"
5,Total,"33,092","S/ 521,955,198"


# Brecha con ordenamiento nacional

Criterios de ordenamiento:
1) Los **nombrados excedentes se reubican dentro de su propia UGEL a una IIEE con requerimiento:** El argumento de este movimiento es que al ser IIEE con mas de 140 alumnos son similares en sus caracteristicas

2) **Si aun existen nombrados excedentes ya no se reubican a una IIEE con requerimiento:** Esto se explica porque realizar un moviento dentro de la misma region puede ocasionar gastos de transporte asociados

3) **Las plazas contratadas o vacantes excedentes se mueven a nivel nacional una IIEE con requerimiento**

In [12]:
brecha_cr = df_ugel.copy()

for x in personal:
    # Brecha UGEL
    brecha_cr[f'brecha_ugel_{x}'] = brecha_cr[f'exd_{x}_n'] + brecha_cr[f'exd_{x}_c'] - brecha_cr[f'req_{x}']
    
    # Nombrados que no pueden ser reasignados
    brecha_cr[f'nom_exd_rig_{x}'] = brecha_cr[f'exd_{x}_n'] - brecha_cr[f'req_{x}'] 
    brecha_cr[f'nom_exd_rig_{x}'] = np.where(brecha_cr[f'nom_exd_rig_{x}']<0, 0 , brecha_cr[f'nom_exd_rig_{x}'] )

In [42]:
#Brecha regional
for x in personal:
    brecha_cr[f'req1_{x}'] = brecha_cr[f'req_{x}'] - brecha_cr[f'exd_{x}_n'] 
    brecha_cr[f'req1_{x}'] = np.where(brecha_cr[f'req1_{x}']<0,0,brecha_cr[f'req1_{x}'])
    brecha_cr[f'brecha_cr_{x}'] = brecha_cr[f'exd_{x}_c'] - brecha_cr[f'req1_{x}']

region_cr = brecha_cr.groupby('d_dpto').sum()
region_cr[['brecha_cr_pers_limp_mant','brecha_cr_pers_vigilancia','brecha_cr_coord_adm_ie','brecha_cr_oficinista',
           'brecha_cr_secretario']]
for x in personal:
    if x == 'oficinista':
        region_cr[f'costo_brecha_cr_{x}'] = region_cr[f'brecha_cr_{x}']*costo_oficinista
    elif x == 'secretario':
        region_cr[f'costo_brecha_cr_{x}'] = region_cr[f'brecha_cr_{x}']*costo_secretario
    else:
        region_cr[f'costo_brecha_cr_{x}'] = region_cr[f'brecha_cr_{x}']*costo_otros

list_b_cr = ['brecha_cr_pers_limp_mant','brecha_cr_pers_vigilancia','brecha_cr_coord_adm_ie','brecha_cr_oficinista',
           'brecha_cr_secretario']
list_b_cr_costos = ['costo_brecha_cr_pers_limp_mant','costo_brecha_cr_pers_vigilancia','costo_brecha_cr_coord_adm_ie','costo_brecha_cr_oficinista',
           'costo_brecha_cr_secretario']    

region_cr[list_b_cr + list_b_cr_costos]
#region_cr[[list_b_cr, list_b_cr_costos]]

,brecha_cr_pers_limp_mant,brecha_cr_pers_vigilancia,brecha_cr_coord_adm_ie,brecha_cr_oficinista,brecha_cr_secretario,costo_brecha_cr_pers_limp_mant,costo_brecha_cr_pers_vigilancia,costo_brecha_cr_coord_adm_ie,costo_brecha_cr_oficinista,costo_brecha_cr_secretario
d_dpto,,,,,,,,,,
AMAZONAS,-32,-220,-153,-29,2,-500544.0,-3441240.0,-2393226.0,-472584.0,37824.0
ANCASH,115,-503,-360,-152,3,1798830.0,-7867926.0,-5631120.0,-2476992.0,56736.0
APURIMAC,-20,-155,-152,-12,2,-312840.0,-2424510.0,-2377584.0,-195552.0,37824.0
AREQUIPA,47,-456,-327,-23,11,735174.0,-7132752.0,-5114934.0,-374808.0,208032.0
AYACUCHO,-26,-254,-177,-82,3,-406692.0,-3973068.0,-2768634.0,-1336272.0,56736.0
CAJAMARCA,-181,-580,-443,-106,4,-2831202.0,-9072360.0,-6929406.0,-1727376.0,75648.0
CALLAO,34,-490,-210,-85,-14,531828.0,-7664580.0,-3284820.0,-1385160.0,-264768.0
CUSCO,-214,-508,-429,-57,13,-3347388.0,-7946136.0,-6710418.0,-928872.0,245856.0
HUANCAVELICA,17,-99,-103,-25,8,265914.0,-1548558.0,-1611126.0,-407400.0,151296.0


In [27]:
cr_costos = []
cr_personal = []

for x in personal:
    cr_personal.append(f'brecha_cr_{x}')
    cr_costos.append(f'costo_brecha_cr_{x}')

resumen_cr_personal = list(brecha_cr[cr_personal].sum())
resumen_cr_costos = list(brecha_cr[cr_costos].sum())
#personal_resumen = ['Limpieza y mantenimiento','Vigilantes','Coordinador administrativo','Oficinista','Secretario']
#dict_resumen_cr = {'Perfiles': personal_resumen, 'Plazas': resumen_cr_personal, 'Costos': resumen_cr_costos} 
    
#resumen_cr = pd.DataFrame(dict_resumen_cr)
#resumen_cr.loc['5'] = resumen_cr.sum()
#resumen_cr.iloc[5, resumen_cr.columns.get_loc('Perfiles')] = 'Total'

#print('En resumen: ')
#resumen_cr.style.format({'Plazas':'{:20,.0f}','Costos':'S/{:20,.0f}' })


KeyError: "None of [Index(['costo_brecha_cr_pers_limp_mant', 'costo_brecha_cr_pers_vigilancia',\n       'costo_brecha_cr_coord_adm_ie', 'costo_brecha_cr_oficinista',\n       'costo_brecha_cr_secretario'],\n      dtype='object')] are in the [columns]"

In [28]:
cr_costos

['costo_brecha_cr_pers_limp_mant',
 'costo_brecha_cr_pers_vigilancia',
 'costo_brecha_cr_coord_adm_ie',
 'costo_brecha_cr_oficinista',
 'costo_brecha_cr_secretario']

In [14]:

b_cr = np.absolute(region_cr.sum())



In [19]:
for x in personal:
    print(f'{x}: ',np.absolute(region_cr[f'brecha_cr_{x}'].sum()))

pers_limp_mant:  2976
pers_vigilancia:  14275
coord_adm_ie:  8203
oficinista:  3499
secretario:  57


In [20]:
for x in personal:
    print(f'{x}: ',region_cr[f'brecha_cr_{x}'].sum())

pers_limp_mant:  -2976
pers_vigilancia:  -14275
coord_adm_ie:  -8203
oficinista:  -3499
secretario:  -57


In [15]:
b_cr



pers_limp_mant_n               17147
pers_limp_mant_c                3799
pers_limp_mant                 20946
opt_pers_limp_mant             23119
exd_pers_limp_mant              3570
req_pers_limp_mant              5743
pers_vigilancia_n                 27
pers_vigilancia_c               1514
pers_vigilancia                 1541
opt_pers_vigilancia            15816
exd_pers_vigilancia                6
req_pers_vigilancia            14281
coord_adm_ie_n                   521
coord_adm_ie_c                    88
coord_adm_ie                     609
opt_coord_adm_ie                8812
exd_coord_adm_ie                 152
req_coord_adm_ie                8355
oficinista_n                    2229
oficinista_c                     665
oficinista                      2894
opt_oficinista                  6266
exd_oficinista                   864
req_oficinista                  4236
secretario_n                    1215
secretario_c                     245
secretario                      1460
o

In [ ]:
b_cr = np.absolute(region_cr['Brecha con Racio'].sum())
costo_cr = b_cr*(12*1150+600+12*(1150*0.09))

In [ ]:
# #Brecha regional
region_cr = brecha_cr.groupby('d_dpto').sum()
region_cr.drop(columns = ['req1','Brecha UGEL con racio'], inplace =  True)
region_cr

In [11]:
b_cr = np.absolute(region_cr['Brecha con Racio'].sum())
costo_cr = b_cr*(12*1150+600+12*(1150*0.09))

print(f'Financiar esta brecha de {b_cr:,} plazas costaria S/{costo_cr:,}')

Financiar esta brecha de 2,976.0 plazas costaria S/46,550,592.0


In [12]:
brecha_cr.drop(columns = ['req1', 'Brecha con Racio'], inplace =  True)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Resultados\Personal de limpieza.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
brecha_sr.to_excel(writer, sheet_name='UGEL sin racio')
region_sr.to_excel(writer, sheet_name='Region sin racio')

brecha_cr.to_excel(writer, sheet_name='UGEL con racio')
region_cr.to_excel(writer, sheet_name='Region con racio')

# Close the Pandas Excel writer and output the Excel file.
writer.save()